# ChaosTrace sur Google Colab

**Date**: 2026-02-09  
**Objectif**: installer ChaosTrace depuis GitHub, lancer un sweep (core), puis tester le mode hybride (optionnel MP et DL), et visualiser les artefacts.

Notes importantes:
- ChaosTrace demande Python **>= 3.11**.
- Les commandes ci-dessous utilisent le module Python (pas de binaire `chaostrace` car il n'y a pas d'entrée console actuellement).



In [ ]:
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("Executable:", sys.executable)

if sys.version_info < (3, 11):
    raise RuntimeError(
        "Python >= 3.11 requis par ChaosTrace. "
        "Dans Colab: Runtime > Change runtime type, puis choisis un runtime récent "
        "(ou utilise un runtime 'fallback' compatible si proposé)."
    )


## 1) Installer ChaosTrace (clone + editable install)

Ce mode est le plus robuste pour Colab et te permet aussi de modifier du code pendant la session.



In [ ]:
import os

REPO = "https://github.com/dalozedidier-dot/ChaosTrace.git"
BRANCH = "main"
DST = "/content/ChaosTrace"

if os.path.exists(DST):
    print("Suppression:", DST)
    !rm -rf "{DST}"

!git clone --depth 1 --branch "{BRANCH}" "{REPO}" "{DST}"

%cd {DST}

!python -m pip install -U pip
!python -m pip install -e ".[dev]"


Option: activer des extras.

- `mp`: Matrix Profile (stumpy)
- `dl`: PyTorch (mode hybride avec modèle entraîné)

Active uniquement ce dont tu as besoin.



In [ ]:
# Option MP
# !python -m pip install -e ".[mp]"

# Option DL (peut être lourd, surtout sans GPU)
# !python -m pip install -e ".[dl]"


## 2) Choisir une source de données

Par défaut on utilise les CSV présents dans le repo:
- `test_data/sample_timeseries.csv` (stable)
- `test_data/sample_timeseries_1_2_drops.csv` (1 à 2 drops)
- `test_data/sample_timeseries_chaotic.csv` (chaotique)

Tu peux aussi monter Drive ou uploader un CSV.



In [ ]:
DATA_STABLE = "test_data/sample_timeseries.csv"
DATA_DROPS = "test_data/sample_timeseries_1_2_drops.csv"
DATA_CHAOTIC = "test_data/sample_timeseries_chaotic.csv"

DATA_PATH = DATA_STABLE
print("DATA_PATH =", DATA_PATH)


In [ ]:
# (Optionnel) Monter Google Drive
# from google.colab import drive
# drive.mount("/content/drive")
# DATA_PATH = "/content/drive/MyDrive/ton_dossier/ton_fichier.csv"


In [ ]:
# (Optionnel) Uploader un fichier CSV directement
# from google.colab import files
# uploaded = files.upload()
# DATA_PATH = list(uploaded.keys())[0]
# print("DATA_PATH =", DATA_PATH)


## 3) Sweep chaos (core)

Génère:
- `metrics.csv`
- `anomalies.csv`
- `fig_phase.png`
- `fig_timeline.png`
- `fig_timeline_inv_var.png`
- `manifest.json`



In [ ]:
import pathlib, shutil, subprocess

OUT_ROOT = pathlib.Path("/content/_ci_out")
OUT_CORE = OUT_ROOT / "colab_core"
if OUT_CORE.exists():
    shutil.rmtree(OUT_CORE)
OUT_CORE.mkdir(parents=True, exist_ok=True)

cmd = [
    sys.executable, "-m", "chaostrace.cli.run_sweep",
    "--input", DATA_PATH,
    "--out", str(OUT_CORE),
]
print(" ".join(cmd))
subprocess.check_call(cmd)

print("\nArtefacts:")
for p in sorted(OUT_CORE.iterdir()):
    print(" -", p.name)


In [ ]:
import pandas as pd, json
from IPython.display import display

metrics_path = OUT_CORE / "metrics.csv"
anoms_path = OUT_CORE / "anomalies.csv"
manifest_path = OUT_CORE / "manifest.json"

dfm = pd.read_csv(metrics_path)
dfa = pd.read_csv(anoms_path)

print("metrics.csv (head)")
display(dfm.head(10))

print("\nanomalies.csv (head)")
display(dfa.head(10))

manifest = json.loads(manifest_path.read_text())
print("\nmanifest.json (keys)")
print(sorted(manifest.keys()))


In [ ]:
from IPython.display import Image, display

for fname in ["fig_phase.png", "fig_timeline.png", "fig_timeline_inv_var.png"]:
    fpath = OUT_CORE / fname
    if fpath.exists():
        print(fname)
        display(Image(filename=str(fpath)))
    else:
        print("Manquant:", fname)


## 4) Mode hybride (sans extras)

Exemple: activation du module causal (VAR) uniquement.



In [ ]:
OUT_HYBRID = OUT_ROOT / "colab_hybrid"
if OUT_HYBRID.exists():
    shutil.rmtree(OUT_HYBRID)
OUT_HYBRID.mkdir(parents=True, exist_ok=True)

cmd = [
    sys.executable, "-m", "chaostrace.cli.run_hybrid",
    "--input", DATA_PATH,
    "--out", str(OUT_HYBRID),
    "--enable-causal",
]
print(" ".join(cmd))
subprocess.check_call(cmd)

print("\nArtefacts:")
for p in sorted(OUT_HYBRID.iterdir()):
    print(" -", p.name)


In [ ]:
import pandas as pd, json
from IPython.display import display

hyb_csv = OUT_HYBRID / "anomalies_hybrid.csv"
hyb_metrics = OUT_HYBRID / "metrics_hybrid.json"

if hyb_csv.exists():
    d = pd.read_csv(hyb_csv)
    print("anomalies_hybrid.csv (head)")
    display(d.head(10))
else:
    print("anomalies_hybrid.csv absent")

if hyb_metrics.exists():
    m = json.loads(hyb_metrics.read_text())
    print("metrics_hybrid.json (keys)")
    print(sorted(m.keys()))
else:
    print("metrics_hybrid.json absent")


In [ ]:
from IPython.display import Image, display

for fname in ["fig_timeline_hybrid.png", "fig_phase_hybrid.png"]:
    fpath = OUT_HYBRID / fname
    if fpath.exists():
        print(fname)
        display(Image(filename=str(fpath)))
    else:
        print("Manquant:", fname)


## 5) Option Matrix Profile (mp)

Cette cellule:
- vérifie la présence de `stumpy`
- installe `.[mp]` si nécessaire
- lance un run hybride MP



In [ ]:
import importlib.util, subprocess

has_stumpy = importlib.util.find_spec("stumpy") is not None
print("stumpy disponible:", has_stumpy)

if not has_stumpy:
    print("Installation de l'extra mp ...")
    %cd {DST}
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", ".[mp]"])


In [ ]:
OUT_HYBRID_MP = OUT_ROOT / "colab_hybrid_mp"
if OUT_HYBRID_MP.exists():
    shutil.rmtree(OUT_HYBRID_MP)
OUT_HYBRID_MP.mkdir(parents=True, exist_ok=True)

cmd = [
    sys.executable, "-m", "chaostrace.cli.run_hybrid",
    "--input", DATA_PATH,
    "--out", str(OUT_HYBRID_MP),
    "--enable-mp",
    "--mp-col", "boat_speed",
]
print(" ".join(cmd))
subprocess.check_call(cmd)

print("\nArtefacts:")
for p in sorted(OUT_HYBRID_MP.iterdir()):
    print(" -", p.name)


## 6) Option DL (entraînement minimal + inférence)

Ce bloc est optionnel.
- Il installe `.[dl]` si PyTorch n'est pas présent.
- Il entraîne un modèle très vite (epochs faibles) sur `sample_timeseries_1_2_drops.csv`.
- Il ré-exécute `run_hybrid` avec `--model`.



In [ ]:
import importlib.util, subprocess

has_torch = importlib.util.find_spec("torch") is not None
print("torch disponible:", has_torch)

if not has_torch:
    print("Installation de l'extra dl ...")
    %cd {DST}
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", ".[dl]"])


In [ ]:
# Vérifie GPU (optionnel)
try:
    subprocess.check_call(["nvidia-smi"])
except Exception:
    print("nvidia-smi non disponible (probablement CPU).")


In [ ]:
MODEL_DIR = OUT_ROOT / "model_dir"
if MODEL_DIR.exists():
    shutil.rmtree(MODEL_DIR)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

cmd = [
    sys.executable, "-m", "chaostrace.cli.train_hybrid",
    "--input", DATA_DROPS,
    "--out", str(MODEL_DIR),
    "--cols", "boat_speed,foil_height_m",
    "--window-s", "5",
    "--stride-s", "0.5",
    "--horizon-s", "0.0",
    "--drop-threshold", "0.4",
    "--supervised-epochs", "1",
]
print(" ".join(cmd))
subprocess.check_call(cmd)

print("\nMODEL_DIR contenu:")
for p in sorted(MODEL_DIR.iterdir()):
    print(" -", p.name)


In [ ]:
OUT_HYBRID_DL = OUT_ROOT / "colab_hybrid_dl"
if OUT_HYBRID_DL.exists():
    shutil.rmtree(OUT_HYBRID_DL)
OUT_HYBRID_DL.mkdir(parents=True, exist_ok=True)

cmd = [
    sys.executable, "-m", "chaostrace.cli.run_hybrid",
    "--input", DATA_DROPS,
    "--out", str(OUT_HYBRID_DL),
    "--model", str(MODEL_DIR),
]
print(" ".join(cmd))
subprocess.check_call(cmd)

print("\nArtefacts:")
for p in sorted(OUT_HYBRID_DL.iterdir()):
    print(" -", p.name)


In [ ]:
from IPython.display import Image, display

for fname in ["fig_timeline_hybrid.png", "fig_phase_hybrid.png"]:
    fpath = OUT_HYBRID_DL / fname
    if fpath.exists():
        print(fname)
        display(Image(filename=str(fpath)))


## 7) Exporter les résultats

Zippe `_ci_out` puis propose un téléchargement.



In [ ]:
import os, shutil
from google.colab import files

zip_path = "/content/chaostrace_colab_outputs.zip"
if os.path.exists(zip_path):
    os.remove(zip_path)

shutil.make_archive("/content/chaostrace_colab_outputs", "zip", root_dir=str(OUT_ROOT))
print("ZIP prêt:", zip_path)
files.download(zip_path)
